In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from urlparse import urlparse
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold

In [2]:
us_company = pd.read_csv('us_company.csv')
yahoo_fin = pd.read_csv('YahooFin.csv')

# The goal is to pickup the domain keyword from either email or domain and try to join the two data sources.
us_company['domain_keyword'] = us_company['CompanyEmailDomain'].apply(lambda x: x.lower().replace('.com', ''))
us_company['log_EMPTOTAL'] = us_company['EMPTOTAL'].apply(lambda x: np.log10(x))
yahoo_fin['domain_keyword'] = yahoo_fin['Domain'].apply(
    lambda x: np.nan if pd.isnull(x) else urlparse(x).netloc.lower().replace('www.', '').split('.')[0])

# Convert a few string columns to numerical columns.
def cap_to_num(str):
    if pd.isnull(str):
        return np.nan
    base_str = str.replace('$', '').replace(',', '')
    if base_str.find('B') > 0:
        base_num = 1000000000
    elif base_str.find('M') > 0:
        base_num = 1000000
    elif base_str.find('K') > 0:
        base_num = 1000
    else:
        base_num = 1
    return float(base_str.replace('B', '').replace('M', '')) * base_num

def employee_convert(employee_str):
    if pd.isnull(employee_str):
        return np.nan
    employee_str_base = employee_str.replace(',', '').replace("\"", '')
    return int(employee_str_base)
                                             

# The market cap for two companies are a little bit too small.
yahoo_fin['market_cap'] = yahoo_fin['MarketCap'].apply(cap_to_num)
yahoo_fin['EMPTOTAL'] = yahoo_fin['Num_Employees'].apply(employee_convert)

In [ ]:
us_company.head()

In [ ]:
yahoo_fin.head()

In [ ]:
ax = sns.distplot(us_company['log_EMPTOTAL'], norm_hist=False, kde=False)
ax.set_title('Histogram of Employee Count')
ax.set_xlabel('log10(Employee Count)')
ax.set_ylabel('Company Count')
plt.show()

In [ ]:
top_industry = us_company.groupby('industry')['EMPTOTAL'].count().\
    reset_index().sort_values(by='EMPTOTAL', ascending=False, inplace=False).industry.tolist()[:7]
us_company['top_industry'] = us_company['industry'].apply(lambda x: x if x in top_industry else 'Other')
ax = sns.violinplot(x='top_industry', y='log_EMPTOTAL', data=us_company)
ax.set_xticklabels(ax.get_xticklabels(), rotation=70)
ax.set_title('Violin Plot of Employee Count by Industry')
ax.set_xlabel('Industry')
ax.set_ylabel('log10(Employee Count)')
plt.show()

In [20]:
# Let's try to estimate employee count given other features.
# Assumptions:
# 1. LOCALSALES and USSALES are accurate.
rf_industry = us_company.groupby('industry')['EMPTOTAL'].count().\
    reset_index().sort_values(by='EMPTOTAL', ascending=False, inplace=False).iloc[:10]['industry'].tolist()
rf_vertical = us_company.groupby('vertical')['EMPTOTAL'].count().\
    reset_index().sort_values(by='EMPTOTAL', ascending=False, inplace=False).iloc[:20]['vertical'].tolist()
rf_category = us_company.groupby('category')['EMPTOTAL'].count().\
    reset_index().sort_values(by='EMPTOTAL', ascending=False, inplace=False).iloc[:20]['category'].tolist()

def filter_top_list(x, top_list):
    if pd.isnull(x) or x not in top_list:
        return 'Other'
    else:
        return x
    
us_company['rf_industry'] = us_company['industry'].apply(lambda x: filter_top_list(x, top_list=rf_industry))
us_company['rf_vertical'] = us_company['vertical'].apply(lambda x: filter_top_list(x, top_list=rf_vertical))
us_company['rf_category'] = us_company['category'].apply(lambda x: filter_top_list(x, top_list=rf_category))
us_company['log_lcoalsales'] = us_company['LOCALSALES'].apply(lambda x: np.log(x + 0.0001))
us_company['log_ussales'] = us_company['USSALES'].apply(lambda x: np.log(x + 0.0001))

feature_list =[us_company[['log_lcoalsales', 'log_ussales']]]
for cal_col in ['rf_industry', 'rf_vertical', 'rf_category']:
    feature_list.append(pd.get_dummies(us_company[cal_col], prefix=cal_col))
feature = pd.concat(feature_list, axis=1)
Y = np.log(us_company['EMPTOTAL'])

In [22]:
RF_model = RandomForestRegressor(n_estimators=800)
kf = KFold(n_splits=10, shuffle=True)
result = []
for train_index, test_index in kf.split(feature):
    X_train, X_test = feature.iloc[train_index], feature.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    RF_model.fit(X_train, y_train)
    y_pred = RF_model.predict(X_test)
    result.append({
        'train_index': train_index,
        'test_index': test_index,
        'y_train': y_train,
        'y_pred': y_pred,
        'MAE_log': np.mean(abs(y_test - y_pred)),
        'MAE': np.mean(abs(np.exp(y_test) - np.exp(y_pred))),
        'RMSE_log': np.sqrt(np.mean((y_test - y_pred) ** 2)),
        'RMSE': np.sqrt(np.mean((np.exp(y_test) - np.exp(y_pred)) ** 2))
    })

In [25]:
np.mean([x['MAE_log'] for x in result]), np.mean([x['MAE'] for x in result])

(0.9710466259071797, 2767.4921800668226)

In [26]:
np.mean([x['RMSE_log'] for x in result]), np.mean([x['RMSE'] for x in result])

(1.3687840541884628, 12365.199759924619)